<a href="https://colab.research.google.com/github/anildesh123/Halide/blob/master/nemo_agent_code_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  NeMo Agent Toolkit — Hello World (Stable Colab Tutorial)

This notebook runs the [NeMo Agent Toolkit](https://github.com/NVIDIA/NeMo-Agent-Toolkit)'s Hello World example using a ReAct agent, Wikipedia tool, and NVIDIA's hosted LLaMA-3.1 via NIM.

**You'll learn:**
- What a ReAct agent is
- How it uses tools + LLMs to solve queries
- How to run the whole setup inside Colab

 You must have a [NVIDIA API key](https://build.nvidia.com) to run this example.


In [ ]:
# ✅ Step 1: Clean any broken or cached directories (safe restart)
import os
os.chdir("/content")
!rm -rf /content/aiqtoolkit


In [ ]:
# ✅ Step 2: Clone NeMo Agent Toolkit and move into it
!git clone https://github.com/NVIDIA/NeMo-Agent-Toolkit.git /content/aiqtoolkit
import os
os.chdir("/content")


Cloning into '/content/aiqtoolkit'...
remote: Enumerating objects: 8570, done.
remote: Counting objects: 100% (363/363), done.
remote: Compressing objects: 100% (251/251), done.
remote: Total 8570 (delta 206), reused 134 (delta 107), pack-reused 8207 (from 3)
Receiving objects: 100% (8570/8570), 37.21 MiB | 22.18 MiB/s, done.
Resolving deltas: 100% (4317/4317), done.
Filtering content: 100% (64/64), 20.78 MiB | 11.69 MiB/s, done.


In [ ]:
# ✅ Step 3: Install `uv` and create virtual environment
!pip install -q uv
!uv venv --seed .venv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.8/18.8 MB 79.0 MB/s eta 0:00:00
Using CPython 3.11.13 interpreter at: /usr/bin/python3
Creating virtual environment with seed packages at: .venv
 + pip==25.2
 + setuptools==80.9.0
 + wheel==0.45.1
Activate with: source .venv/bin/activate


In [ ]:
# ✅ Step 4: Activate env and install toolkit + all extras
%%bash
cd /content/aiqtoolkit
source .venv/bin/activate
uv sync --all-groups --all-extras

bash: line 2: .venv/bin/activate: No such file or directory
Using CPython 3.12.11
Creating virtual environment at: .venv
Resolved 521 packages in 191ms
   Building aiqtoolkit-ragaai @ file:///content/aiqtoolkit/packages/aiqtoolkit_ragaai
   Building aiqtoolkit-agno @ file:///content/aiqtoolkit/packages/aiqtoolkit_agno
   Building aiq-alert-triage-agent @ file:///content/aiqtoolkit/examples/advanced_agents/alert_triage_agent
   Building aiq-email-phishing-analyzer @ file:///content/aiqtoolkit/examples/evaluation_and_profiling/email_phishing_analyzer
   Building aiq-plot-charts @ file:///content/aiqtoolkit/examples/custom_functions/plot_charts
   Building aiq-profiler-agent @ file:///content/aiqtoolkit/examples/advanced_agents/profiler_agent
   Building aiq-semantic-kernel-demo @ file:///content/aiqtoolkit/examples/frameworks/semantic_kernel_demo
   Building aiq-simple-calculator @ file:///content/aiqtoolkit/examples/getting_started/simple_calculator
   Building aiq-simple-calculator-eva

In [ ]:
# ✅ Step 5: Set your NVIDIA API key (required for NIM LLM)
api_key = "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" # 🔒 Replace with real key
with open("/content/aiqtoolkit/env.sh", "w") as f:
    f.write(f"export NVIDIA_API_KEY={api_key}\n")

In [ ]:
# ✅ Step 6: Write the agent config (workflow.yaml)
%%bash
cd /content/aiqtoolkit
cat <<EOF > workflow.yaml
functions:
  wikipedia_search:
    _type: wiki_search
    max_results: 2

llms:
  nim_llm:
    _type: nim
    model_name: meta/llama-3.1-70b-instruct
    temperature: 0.0

workflow:
  _type: react_agent
  tool_names: [wikipedia_search]
  llm_name: nim_llm
  verbose: true
  parse_agent_response_max_retries: 3
EOF

### 🧠 What Does the Config Do?
- `functions`: Defines the Wikipedia tool the agent can use
- `llms`: Uses the LLaMA 3.1-70b model hosted by NVIDIA NIM
- `workflow`: Specifies a `react_agent`, which reasons + acts via the tool
- `verbose`: Enables detailed tracing of agent thoughts/actions

In [ ]:
# ✅ Step 7: Run the Hello World agent
%%bash
cd /content/aiqtoolkit
source .venv/bin/activate
source env.sh
aiq run --config_file workflow.yaml --input "Write a python code to reverse a string"


Configuration Summary:
--------------------
Workflow Type: react_agent
Number of Functions: 1
Number of LLMs: 1
Number of Embedders: 0
Number of Memory: 0
Number of Object Stores: 0
Number of Retrievers: 0
Number of ITS Strategies: 0
Number of Authentication Providers: 0



2025-07-31 09:10:47,880 - aiq.runtime.loader - WARNING - Loading module 'aiq_profiler_agent.register' from entry point 'aiq_profiler_agent' took a long time (1500.620604 ms). Ensure all imports are inside your registered functions.
2025-07-31 09:10:48,012 - aiq.runtime.loader - WARNING - Loading module 'aiq.plugins.agno.register' from entry point 'aiq_agno' took a long time (131.444454 ms). Ensure all imports are inside your registered functions.
2025-07-31 09:10:48,030 - aiq.data_models.discovery_metadata - WARNING - Package metadata not found for simple_auth
2025-07-31 09:10:51,271 - aiq.runtime.loader - WARNING - Loading module 'aiq_alert_triage_agent.register' from entry point 'aiq_alert_triage_agent' took a long time (3047.279119 ms). Ensure all imports are inside your registered functions.
2025-07-31 09:10:51,559 - aiq.runtime.loader - WARNING - Loading module 'aiq.experimental.inference_time_scaling.register' from entry point 'aiq_inference_time_scaling' took a long time (154.53

🎉 If everything is configured properly, the agent will:
1. Think about how to solve the query
2. Use the Wikipedia search tool
3. Call LLaMA 3.1 to formulate a response
4. Print the answer in console
